## Create Data ver 2.0

@Juran 2021.11.28 최종수정

- 미디어파이프 7, 6, 5번(검지손가락) 랜드마크의 각도 정보 계산하여 데이터셋 생성
- 미디어파이프 설치상의 문제로 google colab 이용
- colab에서 웹캠 사용 불가 문제로 영상 촬영하여 업로드 후 이용

### Google Colab File Upload & Install Library

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 동영상 2021. 11. 23. 오전 9.19.MOV to 동영상 2021. 11. 23. 오전 9.19.MOV
Saving 동영상 2021. 11. 23. 오전 9.21.MOV to 동영상 2021. 11. 23. 오전 9.21.MOV
User uploaded file "동영상 2021. 11. 23. 오전 9.19.MOV" with length 33470015 bytes
User uploaded file "동영상 2021. 11. 23. 오전 9.21.MOV" with length 33421906 bytes


In [4]:
!pip install mediapipe

     |████████████████████████████████| 32.8 MB 1.7 MB/s 


### 라이브러리 import 및 미디어파이프 설정

In [5]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import time

In [8]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

### openCV & mediapipe 이용하여 검지손가락 각도 정보 추출

#### 클릭 모션

In [34]:
cap = cv2.VideoCapture('click.MOV') # video open

data_click = pd.DataFrame()
while cap.isOpened():
  ret, img = cap.read()
  if img is None:
    break
  img = cv2.flip(img, 1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  result = hands.process(img)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

  index_point = [[8, 7, 6], [7, 6, 5], [6, 5, 0]]
  point_info = np.zeros((21, 2))
  if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:
      
      for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y]
      
      angles = []
      for point in index_point:
        # 검지손가락 포인트 각도 게산
        ba = point_info[point[0]] - point_info[point[1]]
        bc = point_info[point[2]] - point_info[point[1]]
        cosine_angle = np.dot(ba,bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) 
        pAngle = np.degrees(angle)
        angles.append(pAngle)

      # 라벨 추가
      angles.append(1) # 1 - click

      data_click = data_click.append([angles])

In [35]:
data_click.head()

,0,1,2,3
0,175.231467,176.680216,167.418349,1
0,19.777660,178.893423,163.803281,1
0,158.242057,176.917871,165.684800,1
0,157.850556,175.012657,167.235614,1
0,162.999738,177.413467,165.436078,1


In [36]:
data.tail()

,0,1,2,3
0,22.597918,166.771396,176.578798,1
0,24.050054,166.343032,175.539965,1
0,24.852808,164.162450,176.286538,1
0,25.765116,164.359182,175.603992,1
0,22.199485,166.195664,176.102472,1


In [37]:
data.to_csv("dataset_click.csv")

#### 클릭이 아닌 모션

In [38]:
cap = cv2.VideoCapture('none.MOV') # video open

data_none = pd.DataFrame()
while cap.isOpened():
  ret, img = cap.read()
  if img is None:
    break

  img = cv2.flip(img, 1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  result = hands.process(img)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

  index_point = [[8, 7, 6], [7, 6, 5], [6, 5, 0]]
  point_info = np.zeros((21, 2))
  if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:
      
      for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y]
      
      angles = []
      for point in index_point:
        # 검지손가락 포인트 각도 게산
        ba = point_info[point[0]] - point_info[point[1]]
        bc = point_info[point[2]] - point_info[point[1]]
        cosine_angle = np.dot(ba,bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) 
        pAngle = np.degrees(angle)
        angles.append(pAngle)

      # 라벨 추가
      angles.append(0) # 0 - none

      data_none = data_none.append([angles])

In [39]:
data.to_csv("dataset_none.csv")